### Chapter 13: Constrained Random Verification

Constrained Random Verification (CRV) is one of the most powerful features of SystemVerilog for creating comprehensive testbenches. It allows you to generate random stimulus while maintaining control over the test scenarios through constraints.

#### Introduction to Random Variables

SystemVerilog provides two types of random variables:
- `rand` - Random variable
- `randc` - Random cyclic variable

##### Basic Random Variables

```systemverilog
class packet;
    rand bit [7:0] length;     // Random 8-bit value
    rand bit [31:0] data;      // Random 32-bit value
    randc bit [3:0] ptype;     // Random cyclic 4-bit value
    
    // Non-random variables
    bit [15:0] checksum;       // Regular variable
    
    function void post_randomize();
        checksum = calculate_checksum();
    endfunction
endclass
```

##### rand vs randc

```systemverilog
class example;
    rand bit [2:0] rand_var;    // Can repeat values
    randc bit [2:0] randc_var;  // Cycles through all values before repeating
    
    function void display();
        $display("rand_var = %0d, randc_var = %0d", rand_var, randc_var);
    endfunction
endclass

// Test the difference
example ex = new();
for(int i = 0; i < 10; i++) begin
    ex.randomize();
    ex.display();
end
```

#### Constraint Blocks

Constraints define the legal space for random variable generation.

##### Basic Constraint Syntax

```systemverilog
class ethernet_frame;
    rand bit [15:0] length;
    rand bit [7:0] frame_type;
    rand bit [47:0] dest_addr;
    rand bit [47:0] src_addr;
    
    // Constraint block
    constraint valid_length {
        length >= 64;
        length <= 1518;
    }
    
    constraint valid_frame_type {
        frame_type inside {8'h08, 8'h06, 8'h35, 8'h86};
    }
    
    constraint addr_constraint {
        dest_addr != src_addr;  // Source and destination must be different
        dest_addr[0] == 1'b0;   // Unicast destination
    }
endclass
```

##### Multiple Constraints

```systemverilog
class transaction;
    rand bit [31:0] addr;
    rand bit [7:0] data;
    rand bit [3:0] burst_length;
    
    constraint addr_align {
        addr[1:0] == 2'b00;  // Word aligned
    }
    
    constraint addr_range {
        addr >= 32'h1000_0000;
        addr <= 32'h2000_0000;
    }
    
    constraint burst_size {
        burst_length inside {1, 2, 4, 8, 16};
    }
endclass
```

#### Constraint Expressions

SystemVerilog supports various types of constraint expressions.

##### Arithmetic Constraints

```systemverilog
class math_constraints;
    rand int a, b, c;
    
    constraint arithmetic {
        a + b == c;
        a > 0;
        b > 0;
        c < 1000;
    }
    
    constraint modulo {
        a % 4 == 0;  // a must be divisible by 4
    }
    
    constraint power_of_two {
        $countones(b) == 1;  // b must be power of 2
    }
endclass
```

##### Logical Constraints

```systemverilog
class logic_constraints;
    rand bit [7:0] opcode;
    rand bit enable;
    rand bit [3:0] mode;
    
    constraint logic_rules {
        // Implication
        (opcode == 8'hFF) -> (enable == 1'b1);
        
        // Equivalence
        (mode == 4'h0) <-> (enable == 1'b0);
        
        // Complex logical expression
        (opcode inside {8'h10, 8'h20}) && (mode != 4'hF);
    }
endclass
```

##### Set Membership Constraints

```systemverilog
class membership_example;
    rand bit [7:0] priority;
    rand bit [3:0] category;
    
    constraint priority_levels {
        priority inside {1, 2, 4, 8, 16, 32};
    }
    
    constraint category_constraint {
        // Exclude certain values
        !(category inside {4'h0, 4'hF});
        
        // Weighted inside
        category dist {
            [1:3]   := 50,  // Values 1-3 with weight 50 each
            [4:6]   :/ 30,  // Values 4-6 sharing total weight 30
            [7:14]  := 10   // Values 7-14 with weight 10 each
        };
    }
endclass
```

#### Distribution Constraints

Distribution constraints control the probability of random value generation.

##### Basic Distribution

```systemverilog
class packet_generator;
    rand bit [1:0] packet_type;
    rand bit [7:0] packet_size;
    
    constraint packet_distribution {
        // := assigns weight to each value/range
        packet_type dist {
            2'b00 := 40,  // 40% probability
            2'b01 := 30,  // 30% probability
            2'b10 := 20,  // 20% probability
            2'b11 := 10   // 10% probability
        };
        
        // :/ divides total weight among range
        packet_size dist {
            [1:64]    :/ 50,  // Small packets: 50% total
            [65:256]  :/ 30,  // Medium packets: 30% total
            [257:512] :/ 20   // Large packets: 20% total
        };
    }
endclass
```

##### Advanced Distribution Examples

```systemverilog
class advanced_distribution;
    rand bit [31:0] address;
    rand bit [7:0] burst_length;
    rand bit error_inject;
    
    constraint address_dist {
        address dist {
            [32'h0000_0000 : 32'h0FFF_FFFF] := 60,  // Low memory
            [32'h1000_0000 : 32'h1FFF_FFFF] := 30,  // Mid memory
            [32'h2000_0000 : 32'h2FFF_FFFF] := 10   // High memory
        };
    }
    
    constraint burst_dist {
        burst_length dist {
            1       := 40,  // Single beat most common
            [2:4]   :/ 30,  // Short bursts
            [5:8]   :/ 20,  // Medium bursts
            [9:16]  :/ 10   // Long bursts least common
        };
    }
    
    constraint error_rate {
        error_inject dist {
            1'b0 := 95,  // 95% no error
            1'b1 := 5    // 5% error injection
        };
    }
endclass
```

#### Constraint Inheritance

Classes can inherit and modify constraints from parent classes.

##### Basic Inheritance

```systemverilog
class base_packet;
    rand bit [7:0] length;
    rand bit [3:0] ptype;
    
    constraint base_length {
        length >= 8;
        length <= 64;
    }
    
    constraint base_type {
        ptype inside {4'h1, 4'h2, 4'h3};
    }
endclass

class extended_packet extends base_packet;
    rand bit [15:0] extended_field;
    
    // Override parent constraint
    constraint base_length {
        length >= 16;    // More restrictive
        length <= 128;   // Less restrictive
    }
    
    // Add new constraint
    constraint extended_constraint {
        extended_field != 16'h0000;
        extended_field != 16'hFFFF;
    }
endclass
```

##### Constraint Mode Control

```systemverilog
class configurable_packet;
    rand bit [7:0] data;
    rand bit [3:0] cmd;
    
    constraint data_range {
        data inside {[1:100]};
    }
    
    constraint cmd_constraint {
        cmd != 4'h0;
        cmd != 4'hF;
    }
    
    function void configure_for_debug();
        // Disable constraints for debug mode
        data_range.constraint_mode(0);
        cmd_constraint.constraint_mode(0);
    endfunction
    
    function void configure_for_normal();
        // Re-enable constraints
        data_range.constraint_mode(1);
        cmd_constraint.constraint_mode(1);
    endfunction
endclass
```

#### solve...before Constraints

The `solve...before` construct controls the order of variable solving.

##### Basic solve...before

```systemverilog
class ordered_solving;
    rand bit [7:0] x, y, z;
    
    constraint order_constraint {
        solve x before y;
        solve y before z;
        
        x + y == z;
        x inside {[1:10]};
        y inside {[1:10]};
    }
endclass
```

##### Complex Ordering Example

```systemverilog
class memory_transaction;
    rand bit [31:0] base_addr;
    rand bit [31:0] offset;
    rand bit [31:0] final_addr;
    rand bit [7:0] burst_length;
    rand bit [2:0] burst_size;
    
    constraint solve_order {
        solve base_addr before offset;
        solve offset before final_addr;
        solve burst_size before burst_length;
    }
    
    constraint address_calc {
        final_addr == base_addr + offset;
        base_addr % 4096 == 0;  // Page aligned
        offset < 4096;          // Within page
    }
    
    constraint burst_relationship {
        burst_length <= (8 << burst_size);  // burst_size determines max length
        burst_length inside {1, 2, 4, 8, 16};
    }
endclass
```

#### Advanced Constraint Techniques

##### Conditional Constraints

```systemverilog
class conditional_packet;
    rand bit [1:0] packet_type;
    rand bit [7:0] length;
    rand bit [15:0] payload_length;
    rand bit has_header;
    
    constraint conditional_logic {
        // If-else constraint
        if (packet_type == 2'b00) {
            length inside {[8:16]};
            has_header == 1'b0;
        } else if (packet_type == 2'b01) {
            length inside {[17:64]};
            has_header == 1'b1;
        } else {
            length inside {[65:255]};
            has_header == 1'b1;
        }
        
        // Implication constraint
        has_header -> (payload_length == length - 8);
        !has_header -> (payload_length == length);
    }
endclass
```

##### Array Constraints

```systemverilog
class array_constraints;
    rand bit [7:0] data[];
    rand int size;
    
    constraint array_size {
        size inside {[4:16]};
        data.size() == size;
    }
    
    constraint array_content {
        foreach(data[i]) {
            data[i] != 8'h00;
            data[i] != 8'hFF;
            if (i > 0) data[i] != data[i-1];  // No consecutive duplicates
        }
    }
    
    constraint array_sum {
        data.sum() with (int'(item)) < 1000;
    }
endclass
```

##### Constraint Functions

```systemverilog
class function_constraints;
    rand bit [7:0] value1, value2, value3;
    
    function bit is_prime(int n);
        if (n < 2) return 0;
        for (int i = 2; i * i <= n; i++) begin
            if (n % i == 0) return 0;
        end
        return 1;
    endfunction
    
    constraint using_functions {
        is_prime(value1);
        value2 == fibonacci(value1);  // Assuming fibonacci function exists
        value3 inside {get_valid_values()};  // Function returning array
    }
endclass
```

#### Practical Examples

##### Complete Test Environment

```systemverilog
class network_packet;
    rand bit [47:0] dest_mac;
    rand bit [47:0] src_mac;
    rand bit [15:0] ether_type;
    rand bit [7:0] payload[];
    rand int payload_size;
    
    // Ethernet constraints
    constraint mac_constraints {
        dest_mac[0] == 1'b0;  // Unicast
        src_mac != dest_mac;
        src_mac != 48'h0;
    }
    
    constraint ether_type_constraint {
        ether_type dist {
            16'h0800 := 60,  // IPv4
            16'h0806 := 20,  // ARP
            16'h86DD := 15,  // IPv6
            16'h8100 := 5    // VLAN
        };
    }
    
    constraint payload_constraint {
        payload_size inside {[46:1500]};  // Ethernet payload range
        payload.size() == payload_size;
        foreach(payload[i]) {
            payload[i] != 8'h00;  // Avoid null bytes
        }
    }
    
    function void post_randomize();
        $display("Generated packet: dest=%h, src=%h, type=%h, size=%0d",
                dest_mac, src_mac, ether_type, payload_size);
    endfunction
endclass

// Test program
program test_random_packets;
    network_packet pkt;
    
    initial begin
        pkt = new();
        
        repeat(10) begin
            if (pkt.randomize()) begin
                // Use the packet
                send_packet(pkt);
            end else begin
                $error("Randomization failed!");
            end
        end
    end
endprogram
```

##### Constraint Debugging

```systemverilog
class debug_constraints;
    rand bit [7:0] a, b, c;
    
    constraint debug_c1 {
        a + b == c;
    }
    
    constraint debug_c2 {
        a inside {[10:20]};
    }
    
    constraint debug_c3 {
        b inside {[30:40]};
    }
    
    constraint debug_c4 {
        c inside {[35:45]};  // This might over-constrain
    }
    
    function void debug_randomization();
        int success_count = 0;
        
        for (int i = 0; i < 100; i++) begin
            if (this.randomize()) begin
                success_count++;
                $display("Success %0d: a=%0d, b=%0d, c=%0d", 
                        success_count, a, b, c);
            end else begin
                $display("Randomization failed at attempt %0d", i);
                // Try disabling constraints one by one to debug
                debug_c4.constraint_mode(0);
                if (this.randomize()) begin
                    $display("Issue was with debug_c4 constraint");
                end
                debug_c4.constraint_mode(1);
            end
        end
        
        $display("Success rate: %0d%%", success_count);
    endfunction
endclass
```

#### Best Practices

##### Constraint Organization
- Group related constraints together
- Use meaningful constraint names
- Document complex constraint logic

##### Performance Considerations
```systemverilog
class performance_tips;
    rand bit [31:0] addr;
    rand bit [7:0] data[];
    
    // Good: Simple, efficient constraint
    constraint addr_align {
        addr[1:0] == 2'b00;
    }
    
    // Avoid: Complex constraint that's hard to solve
    constraint complex_bad {
        foreach(data[i]) {
            foreach(data[j]) {
                if (i != j) data[i] != data[j];  // Very expensive
            }
        }
    }
    
    // Better: More efficient uniqueness constraint
    constraint unique_data {
        foreach(data[i]) {
            foreach(data[j]) {
                (i < j) -> (data[i] != data[j]);
            }
        }
    }
endclass
```

##### Debugging Strategies
- Use constraint_mode() to isolate issues
- Implement good post_randomize() functions
- Add constraint guards for impossible conditions

#### Summary

Constrained Random Verification provides powerful capabilities for generating comprehensive test scenarios:

- **Random Variables**: Use `rand` for general randomization, `randc` for cyclic coverage
- **Constraints**: Define legal value spaces with various expression types
- **Distributions**: Control probability of value generation
- **Inheritance**: Extend and modify constraints in derived classes
- **solve...before**: Control variable solving order
- **Advanced Techniques**: Conditional constraints, arrays, and functions

The key to effective CRV is balancing randomness with meaningful constraints that create realistic and comprehensive test scenarios while maintaining solver efficiency.

This completes Chapter 13 on Constrained Random Verification, providing the foundation for building sophisticated verification environments in SystemVerilog.